In [ ]:
# code to generate yearly summaries of DUNE data volumes from input parameters
# rewritten from the version in the CDR - mainly by using maps of years instead of arrays to make it clearer what is in each year.
# HMS 2022-10-23


import os,sys,string,time,commentjson,datetime, math
from csv import reader
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

DEBUG = False
DRAW = True
import numpy as np

from NumberUtils import dump
from NumberUtils import DrawTex
from NumberUtils import cumulateMap
from NumberUtils import DrawDet
from NumberUtils import DrawType
from NumberUtils import makeArray
from NumberUtils import ToCSV1
from NumberUtils import ToCSV2


In [ ]:
# read in a configfile
configfile = "Parameters_2022-10-23-2040.json"

#if len(sys.argv) > 1:
#  configfile = sys.argv[1]

shortname = configfile.replace(".json","")
if os.path.exists(configfile):
  with open(configfile,'r') as f:
    config = commentjson.load(f)
else:
  print ("no config file",configfile)
  sys.exit(0)

if not "Version" in config or config["Version"] < 5:
  print (" this code expects Version >= 2")
  sys.exit(1)

json_formatted_str = commentjson.dumps(config, indent=2)

In [ ]:
MaxYear = config["MaxYear"]

MinYear = config["MinYear"]
Detectors = config["Detectors"]
if DEBUG:
  Detectors = ["SP","SP2","DP"]
Years = np.array(config["Years"])
if DEBUG:
  Years = Years[0:7]
shortname = shortname.replace("2040","%d"%MaxYear)
print (Years, len(Years))
size = len(Years)

Units = config["Units"]

Detectors = config["Detectors"]

Cap = config["Cap"]

print (Detectors)

CombinedDetectors = config["CombinedDetectors"]

DetectorParameters = list(config["SP"].keys())
if "Comment" in DetectorParameters:
    DetectorParameters.remove("Comment")

TapeLifetimes = config["TapeLifetimes"]

DiskLifetimes = config["DiskLifetimes"]

TapeCopies = config["TapeCopies"]

DiskCopies = config["DiskCopies"]

# this is how far you go back each time you reprocess reco.
Reprocess = config["Reprocess"]

PerYear = config["PerYear"]

StorageTypes = list(TapeCopies.keys())

# plot config
DetColors=config["DetColors"]
DetLines = config["DetLines"]
TypeColors=config["TypeColors"]
TypeLines = config["TypeLines"]

PatternFraction = config["PatternFraction"]

SplitsYear = config["SplitsYear"]
SplitsEarly = config["SplitsEarly"]
SplitsLater = config["SplitsLater"]


In [ ]:
dofirst = ["Events","Test","Sim Events"]

# read in the raw information

Inputs = {}
for det in Detectors:
  Inputs[det]={}
  for type in dofirst:
      Inputs[det][type]={}
      for year in Years:
          Inputs[det][type][year] = float(config[det][type][year-Years[0]])

# fill in other useful arrays
for det in Detectors:
  if DEBUG: print ("Events",det,Inputs[det]["Events"])
  for key in DetectorParameters:
    # skip the ones already done
    if key in dofirst:
      continue
    # sim has its own configuration
    # print ("this is the key",det,key)
    if not "Sim" in key:
      if key in ["CPU","Reco"]:  # if doing reco, do over previous events using memory
            Inputs[det][key] = cumulateMap(Years,Inputs[det]["Events"],Reprocess[det])
            for year in Years:
                Inputs[det][key][year] *= config[det][key]
      else:
        if key == "Raw":
            Inputs[det][key] ={}
            for year in Years:
                Inputs[det][key][year] = Inputs[det]["Events"][year]*config[det][key]
        else:
            continue
    else:
        Inputs[det][key] ={}
        for year in Years:
            Inputs[det][key][year]=Inputs[det]["Sim Events"][year]*config[det][key]

In [ ]:
# make a data file which uses # of events to figure out how big samples are

if PerYear["Reco"]!=PerYear["CPU"]:
    print ("Data growth has to match reprocessing cycles/year")
    PerYear["Reco"] = PerYear["CPU"]
if PerYear["Sim"]!=PerYear["Sim-CPU"]:
    print ("Sim growth has to match reprocessing cycles/year")
    PerYear["Sim"] = PerYear["Sim-CPU"]

Data = {}
dump = open("dump.txt",'w')
#print (Inputs.keys())
fields = list(Inputs["ND"].keys())
print ("fields",fields)
for dtype in fields:
  Data[dtype] = {}
  for det in Inputs.keys():
    Data[dtype][det] = {}
    # this allows you to, say, do 2 passes of reco/year
    for year in Years:
        Data[dtype][det][year] = Inputs[det][dtype][year] * float(PerYear[dtype])
    # compensate for nominal units being millions and TB or singles and MB
    if Units[dtype] == "PB":
        for year in Years:
            Data[dtype][det][year] *= 0.001
    ds = "data %s %s %f\n"%(dtype,det,Data[dtype][det][2022])
    dump.write(ds)

In [ ]:
dtype = "Raw+Test"
Data[dtype] = {}
for det in Inputs.keys():
    Data[dtype][det] = {}
    
    for year in Years:
        Data[dtype][det][year] =  Data["Raw"][det][year]+Data["Test"][det][year]
    # compensate for nominal units being millions and TB or singles and MB



In [ ]:
# impose a cap at Cap on things derived from raw data

dtype = "Raw"

Data["Raw"]["Total"] = {}
for year in Years:
        Data[dtype]["Total"][year] = 0.0
for det in Inputs.keys():
    for year in Years:
        Data[dtype]["Total"][year] +=  Data["Raw"][det][year]
        
dtypes = ["Raw","Reco"]
for dtype in dtypes:
    for det in Inputs.keys():
        #print (dtype,det,2035,1.0,Data[dtype][det][2035] )
        for year in Years:
            cap = Data["Raw"]["Total"][year]/Cap
           # print (dtype,det,year,cap,Data[dtype][det][year] )
            if cap > 1:
                Data[dtype][det][year] /=cap
        #print (dtype,det,2035,cap,Data[dtype][det][2035] )

dtypes = ["Raw","Reco"]
for dtype in dtypes:
    Data[dtype]["Total"] ={}
    for year in Years:
        Data[dtype]["Total"][year] = 0.0
    for det in Inputs.keys():
        for year in Years:
           Data[dtype]["Total"][year]+=  Data[dtype][det][year]        
    

In [ ]:
PlotYears = []
for i in range(MinYear,MaxYear-1):
    PlotYears.append(i)
PlotYears = Years

# draw things

if DRAW:
    DrawDet(shortname,"Events",PlotYears,Data,Inputs.keys(),Units,DetColors,DetLines)
    DrawDet(shortname,"CPU",Years,Data,Inputs.keys(),Units,DetColors,DetLines)
    DrawDet(shortname,"Raw",Years,Data,Inputs.keys(),Units,DetColors,DetLines)
    DrawDet(shortname,"Test",Years,Data,Inputs.keys(),Units,DetColors,DetLines)
    DrawDet(shortname,"Sim Events",Years,Data,Inputs.keys(),Units,DetColors,DetLines)
    DrawDet(shortname,"Sim",Years,Data,Inputs.keys(),Units,DetColors,DetLines)
    DrawDet(shortname,"Reco",Years,Data,list(Inputs.keys())+["Total"],Units,DetColors,DetLines)
    DrawDet(shortname,"Raw",Years,Data,list(Inputs.keys())+["Total"],Units,DetColors,DetLines)

In [ ]:
Data.pop("Raw+Test")  # need to remove as it should not go in totals. 

In [ ]:
# merge far detector info

print ("Data keys",Data.keys())

for dtype in Data.keys():
   
    det = "ProtoDUNEs" 
    Data[dtype][det] = {}
    for year in Years:  
        Data[dtype][det][year] = Data[dtype]["SP"][year] + Data[dtype]["DP"][year] + Data[dtype]["SP2"][year] + Data[dtype]["PDVD"][year]

    Data[dtype].pop("SP")
    Data[dtype].pop("SP2")
    Data[dtype].pop("DP")
    Data[dtype].pop("PDVD")

In [ ]:
# merge far detector into "FDs
for dtype in Data.keys():
  det = "FDs"
  Data[dtype][det] =  {}
  for year in Years:  
      Data[dtype][det][year] = Data[dtype]["HD"][year] + Data[dtype]["VD"][year]
  Data[dtype].pop("HD")
  Data[dtype].pop("VD")


In [ ]:
# make a total CPU category

Data["Total-CPU"]={}

for det in CombinedDetectors:
    Data["Total-CPU"][det] =  {}
    for year in Years:
        Data["Total-CPU"][det][year] = Data["CPU"][det][year] + Data["Sim-CPU"][det][year]
    #print(det,Data["Total-CPU"][det])

# do I want to remove CPU as a category? 
Data.pop("CPU")

In [ ]:
# make totals across categories. 

DataTypes = list(Data.keys())

for dt in DataTypes:
    Data[dt]["Total"] = {}
    for year in Years:
        Data[dt]["Total"][year]=0.0
    for k in Data[dt].keys():
        if k == "Total":
          continue  
        for year in Years:
            Data[dt]["Total"][year] += Data[dt][k][year]
    

In [ ]:
Data["Total-CPU"]["Analysis"]= {}
for year in Years:
    Data["Total-CPU"]["Analysis"][year]= 0.0
    
for det in config["Analysis"]["Add"]:
    for year in Years:
    # scale allows you to change the scale of analysis relative to reconstruction and simulations
          Data["Total-CPU"]["Analysis"][year]+= Data["Total-CPU"][det][year]*config["Analysis"]["Scale"]
    
# and put it in the total

for year in Years:
    Data["Total-CPU"]["Total"][year] += Data["Total-CPU"]["Analysis"][year]



In [ ]:
# and make a special data type for cores

Data["Cores"] = {}
Data["HS06"] = {}
Data["WALL"] = {}
 
MHrsPerYear = 1000000./365./24.
for k in Data["Total-CPU"].keys():
    if "MARS" not in k :
        efficiency = config["Cores"]["Efficiency"]
    else:
        efficiency = 1

    scaleTo2020 = config["Cores"]["2020Units"]
    Data["Cores"][k]={}
    Data["HS06"][k]={}
    Data["WALL"][k]={}
    for year in Years:
        Data["Cores"][k][year] = Data["Total-CPU"][k][year]*MHrsPerYear/efficiency/scaleTo2020
        Data["HS06"][k][year] = Data["Total-CPU"][k][year]*MHrsPerYear/efficiency/scaleTo2020*config["kHEPSPEC06PerCPU"]
        Data["WALL"][k][year] = Data["Total-CPU"][k][year]*MHrsPerYear/efficiency/scaleTo2020


In [ ]:
ToCSV2("CoresbyDet","Cores",Data,Units)

In [ ]:
# work out split between different institutions

Splits = {}
for f in SplitsEarly:
    Splits[f] = {}
    for t in SplitsEarly[f]:
        Splits[f][t] = {}
        for loc in SplitsEarly[f][t]: 
            Splits[f][t][loc] = {}
            #print (f,t,Splits[f][t],Splits[f][t][0])
    
            for y in Years:
                if y < SplitsYear:
                    Splits[f][t][loc][y]=SplitsEarly[f][t][loc]
                else:
                    Splits[f][t][loc][y]=SplitsLater[f][t][loc]

In [ ]:
# now do some cumulative work.  Stuff stays on tape/disk for different amounts of time and we have multiple copies

Data["Total"] = {}
Data["Collab"] = {}
Data["FNAL"] = {}
Data["CERN"] = {}
Data["Total"]["Cumulative Tape"] = {}
Data["Total"]["Cumulative Disk"] = {}
Data["FNAL"]["Cumulative Tape"] = {}
Data["FNAL"]["Cumulative Disk"] = {}
Data["CERN"]["Cumulative Tape"] = {}
Data["CERN"]["Cumulative Disk"] = {}
Data["Collab"]["Cumulative Tape"] = {}
Data["Collab"]["Cumulative Disk"] = {}


for year in Years:
    Data["Total"]["Cumulative Tape"][year] = 0.0
    Data["Total"]["Cumulative Disk"][year] = 0.0

for k in StorageTypes:
    Data[k]["Tape"] = {}
    Data[k]["Disk"] = {}
    for year in Years:
        Data[k]["Tape"][year] = Data[k]["Total"][year]*TapeCopies[k]
        Data[k]["Disk"][year] = Data[k]["Total"][year]*DiskCopies[k]
    Data[k]["Cumulative Tape"] = cumulateMap(Years,Data[k]["Tape"],TapeLifetimes[k])
    Data[k]["Cumulative Disk"] = cumulateMap(Years,Data[k]["Disk"],DiskLifetimes[k])
    
    for year in Years:
        Data["Total"]["Cumulative Tape"][year] += Data[k]["Cumulative Tape"][year]
        Data["Total"]["Cumulative Disk"][year] += Data[k]["Cumulative Disk"][year]
        
for loc in Splits["Disk"]["Raw"]:
    for year in Years:
        Data[loc]["Cumulative Disk"][year] = 0.0
        Data[loc]["Cumulative Tape"][year] = 0.0       
        for k in StorageTypes:
              Data[loc]["Cumulative Disk"][year] += Data[k]["Cumulative Disk"][year]*Splits["Disk"][k][loc][year]
              Data[loc]["Cumulative Tape"][year] += Data[k]["Cumulative Tape"][year]*Splits["Tape"][k][loc][year]


In [ ]:
ToCSV1("Disk_by_location","Total","Cumulative Disk",Data,Units)

In [ ]:
# now do some plots

Types = ["ProtoDUNEs","FDs","ND", "Analysis","Total"]

cpuactual = []
coreactual = []
wallactual = []



DrawDet(shortname,"Total-CPU",Years,Data,Types,Units,DetColors,DetLines,cpuactual)
DrawDet(shortname,"Cores",Years,Data,Types,Units,DetColors,DetLines,coreactual)
DrawDet(shortname,"WALL",Years,Data,Types,Units,DetColors,DetLines,wallactual)
DrawDet(shortname,"HS06",Years,Data,Types,Units,DetColors,DetLines,wallactual)

In [ ]:

tapepoints = np.zeros(len(Years))
diskpoints = np.zeros(len(Years))

DrawType(shortname,"Tape",Years,Data,StorageTypes+["Total"],Units,TypeColors,TypeLines,None,None)
DrawType(shortname,"Cumulative Tape",Years,Data,StorageTypes+["Total"],Units,TypeColors,TypeLines,None,None)
DrawType(shortname,"Cumulative Disk",Years,Data,StorageTypes+["Total"],Units,TypeColors,TypeLines,None,None)
#DrawType(shortname,"Cumulative Disk",Years,Data,StorageTypes+["Total"],Units,TypeColors,TypeLines,None,None)

